In [1]:
import pyomo.environ as pyo
import numpy as np
from pyomo.contrib.sensitivity_toolbox.sens import get_dsdp, get_dfds_dcds
#from idaes.apps.uncertainty_propagation.uncertainties import propagate_uncertainty
#from idaes.apps.uncertainty_propagation.sens import get_dsdp

In [2]:
def create_model():
    ### Create optimization model
    m = pyo.ConcreteModel()
    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    # Define variables
    m.x1 = pyo.Var(bounds=(0,10))
    m.x2 = pyo.Var(bounds=(0,10))
    m.x3 = pyo.Var(bounds=(0,10))

    # Define parameters
    #m.p1 = pyo.Param(initialize=10, mutable=True)
    #m.p2 = pyo.Param(initialize=5, mutable=True)
    # Important Tip: The uncertain parameters need to be defined at Pyomo variables
    m.p1 = pyo.Var(initialize=10)
    m.p2 = pyo.Var(initialize=5)
    # Important Tip: We fix these to first solve with Ipopt. We unfix below
    # before using the uncertainty propagation toolbox.
    m.p1.setlb(10)
    m.p1.setub(10)
    m.p2.setlb(5)
    m.p2.setub(5)

    # Define constraints
    m.con1 = pyo.Constraint(expr=m.x1 + m.x2-m.p1==0)
    m.con2 = pyo.Constraint(expr=m.x2 + m.x3-m.p2==0)

    # Define objective
    m.obj = pyo.Objective(expr=m.p1*m.x1+ m.p2*(m.x2**2) + m.p1*m.p2, sense=pyo.minimize)
    
    return m 


In [5]:
variable_name = ['p1','p2']
#variable_name = ['x1','x2','x3','p1','p2']

mod = create_model()



#gradient_f, gradient_c, line_dic =  get_sensitivity(mod, variable_name)
gradient_f, gradient_c, col,row, line_dic = get_dfds_dcds(mod, variable_name, tee=True)
## Run sensitivity toolbox
#sigma_p = np.array([[1, 0], [0, 1]])
#results = propagate_uncertainty(mod,{'p1':10, 'p2':5}, sigma_p, variable_name)

dsdp_re, col = get_dsdp(mod, variable_name, {'p1':10, 'p2':5}, tee=True)

Ipopt 3.13.3: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.3, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        3
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequ

In [6]:
var_idx = np.array([True,True,False,False,True])
print(results.dsdp.toarray()[var_idx,:])

NameError: name 'results' is not defined

In [7]:

dsdp_array = dsdp_re.toarray().T
print(dsdp_array)
print(sparse.csr_matrix(dsdp_array))
print(col)

[[ 9.00000000e-01  2.00000000e-01]
 [ 1.00000000e-01 -2.00000000e-01]
 [ 1.00000000e+00  5.55111512e-17]
 [ 0.00000000e+00  1.00000000e+00]
 [-1.00000000e-01  1.20000000e+00]]
  (0, 0)	0.8999999997992688
  (0, 1)	0.19999999995630358
  (1, 0)	0.10000000020073119
  (1, 1)	-0.19999999995630352
  (2, 0)	1.0
  (2, 1)	5.551115123125783e-17
  (3, 1)	0.9999999999999999
  (4, 0)	-0.10000000020073119
  (4, 1)	1.1999999999563031
['x1', 'x2', 'p1', 'p2', 'x3']


In [ ]:
print((mod.x1()))
print((mod.x2()))
print((mod.x3()))
print((mod.p1()))
print((mod.p2()))
print((mod.obj()))

In [8]:
print(gradient_f)

[10. 10. 14. 11.  0.]


In [9]:
print(gradient_c)

  (0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	-1.0
  (1, 1)	1.0
  (1, 3)	-1.0


In [ ]:
print(col)
print(row)
print(line_dic)

In [ ]:
### Inspect solution
print("Numeric solution:")
print("x1 =",m.x1())
print("x2 =",m.x2())
print("x3 =",m.x3())
print(m.dual.display())


In [ ]:
def derivative_numerical(par):
    ### Create optimization model
    m = pyo.ConcreteModel()

    # Define variables
    m.x1 = pyo.Var()
    m.x2 = pyo.Var()
    m.x3 = pyo.Var()

    # Define parameters
    m.p1 = pyo.Param(initialize=par[0])
    m.p2 = pyo.Param(initialize=par[1])

    # Define constraints
    m.con1 = pyo.Constraint(expr=m.x1 + m.x2-m.p1==0)
    m.con2 = pyo.Constraint(expr=m.x2 + m.x3-m.p2==0)
    m.add = pyo.Constraint(expr=m.p1*m.x1+ m.p2*(m.x2**2) + m.p1*m.p2 == 145.0)
    
    return m 

In [ ]:
#scenario = [[10.1,5], [9.9,5], [10,5.05], [10,4.95]]
scenario = [[10,5], [9.9,5], [10,5], [10,4.95]]
#scenario = [[10,5]]

x1_store = []
x2_store = []
x3_store = []
for j in scenario:
    kitty = derivative_numerical(j)
    solver = SolverFactory('ipopt')
    solver.solver_options = 'ma57'
    solver.solve(kitty,tee=True)
    
    print(value(kitty.p1))
    print(value(kitty.p2)
    
    x1_store.append(value(kitty.x1))
    x2_store.append(value(kitty.x2))
    x3_store.append(value(kitty.x3))

In [ ]:
print(x1_store)
print(x2_store)
print(x3_store)

x1p1 = (x1_store[0]-x1_store[1])/0.1
x1p2 = (x1_store[2]-x1_store[3])/0.05
x2p1 = (x2_store[0]-x2_store[1])/0.1
x2p2 = (x2_store[2]-x2_store[3])/0.05
x3p1 = (x3_store[0]-x3_store[1])/0.1
x3p2 = (x3_store[2]-x3_store[3])/0.05

print(x1p1)
print(x1p2)
print(x2p1)
print(x2p2)
print(x3p1)
print(x3p2)